In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.callbacks import LearningRateScheduler
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import cv2
from keras.models import load_model
%matplotlib inline


## Load Data

In [4]:
# get ids and labels of the train images
train_labels = pd.read_csv("../input/arabic-hwr-ai-pro-intake1/train.csv")
train_labels =train_labels.set_index("id")
train_labels.head()

In [5]:
train_labels.value_counts()

The dataset is perfectly balanced with each letter has 480 training examples

In [6]:
# get ids of the test images
test_ids = pd.read_csv("../input/arabic-hwr-ai-pro-intake1/test.csv")
test_ids.head()

In [7]:
# load train images and there corresponding labels
path = os.path.join("../input/arabic-hwr-ai-pro-intake1/train")
data = []
labels =[]
for img in os.listdir(path):
    labels.append (train_labels.loc[int(os.path.splitext(img)[0])]["label"]-1) # minus one to get labels from 0 to 27
    try:
        img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
        data.append(img_arr)
    except Exception as e:
            print(e)    

In [8]:
# load test images and there corresponding ids
path2 = os.path.join("../input/arabic-hwr-ai-pro-intake1/test")
data_test = []
id_test = []
for img in os.listdir(path2):
    id_test.append(test_ids.loc[int(os.path.splitext(img)[0])]["id"])
    try:
        img_arr = cv2.imread(os.path.join(path2, img))[...,::-1] #convert BGR to RGB format
        data_test.append(img_arr)
    except Exception as e:
            print(e)    

In [8]:
# visualize one of the train images and its label
plt.imshow(data[10000])
plt.title(labels[10000]);

## Data Preprocessing

In [11]:
# split data to train and validation sets
X = np.asarray(data[:])
y = np.asarray(labels[:])
x_train, x_val, y_train, y_val = train_test_split(X,y, test_size = 0.2,stratify=y, random_state = 42)
x_test = np.asarray(data_test[:])
y_train = y_train.reshape(-1,1)
y_val = y_val.reshape(-1,1)

In [32]:

# # building the input vector from the 32x32 pixels
x_train = x_train.reshape(x_train.shape[0], 32, 32, 3)
x_val = x_val.reshape(x_val.shape[0], 32, 32, 3)
x_test = x_test.reshape(x_test.shape[0], 32, 32, 3)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')


# normalizing the data to help with the training
x_train /= 255
x_val /= 255
x_test /= 255


# one-hot encoding using keras' numpy-related utilities
n_classes = 28
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_val = np_utils.to_categorical(y_val, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

## CNN Model

In [33]:
# create a checkpoint to save the best model among epochs on the validation data
checkpoint = ModelCheckpoint(filepath ="./best_weight.h5",
                            monitor='val_accuracy',
                            verbose=1,
                            save_best_only=True,
                            mode='max')

In [34]:
# building a linear stack of layers with the sequential model
model = Sequential()

# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu', input_shape=(32, 32, 3)))

# convolutional layer
model.add(Conv2D(75, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(125, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(28, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

#model.fit(x_train, Y_train, batch_size=128, epochs=100, validation_data=(x_val, Y_val),callbacks=[checkpoint]);


In [35]:
# Data augmentation to create rotated, zoomed and shifted versions of the train images
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [37]:
# training the model
history = model.fit_generator(datagen.flow(x_train,Y_train, batch_size=128),
                              epochs = 80, validation_data = (x_val,Y_val),callbacks=[checkpoint])


## Predictions

In [41]:
# load the best model saved during the training using the checkpoint
optimal_model = load_model('./best_weight.h5')
# recompiling the sequential model
optimal_model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

In [42]:
# use the best model to classify the test images
y_test_predicted = optimal_model.predict(x_test)
y_classes = [np.argmax(y, axis=None, out=None)+ 1 for y in y_test_predicted]


In [43]:
# check some test images manually
plt.imshow(x_test[1])
plt.title(y_classes[1])

In [44]:
# map the predicted classes of the test images to their corresponding ids
y_new = np.zeros((len(test_ids["id"].values),1),dtype = "int")
for i in range(0,len(id_test)):
    for j in range(0,len(test_ids["id"].values)):
        if id_test[i] == test_ids["id"].values[j]:
            y_new[j] = y_classes[i]

In [45]:
y_new[0:5]

In [46]:
test_ids["label"] = y_new
test_ids["label"].value_counts().sort_index()

In [48]:
# # save predictions in a submission file
# test_ids[['id','label']].to_csv('./submission_data_aug1_80epoch.csv', index=False)



In [22]:


# # BUILD CONVOLUTIONAL NEURAL NETWORKS
# nets = 8
# model = [0] *nets

# for j in range(8):
#     model[j] = Sequential()
#     model[j].add(Conv2D(32,kernel_size=5,activation='relu',input_shape=(32,32,1)))
#     model[j].add(MaxPool2D())
#     model[j].add(Dropout(j*0.1))
#     model[j].add(Conv2D(64,kernel_size=5,activation='relu'))
#     model[j].add(MaxPool2D())
#     model[j].add(Dropout(j*0.1))
#     model[j].add(Flatten())
#     model[j].add(Dense(128, activation='relu'))
#     model[j].add(Dropout(j*0.1))
#     model[j].add(Dense(28, activation='softmax'))
#     model[j].compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]);



In [23]:
# # TRAIN NETWORKS
# history = [0] * nets
# names = ["D=0","D=0.1","D=0.2","D=0.3","D=0.4","D=0.5","D=0.6","D=0.7"]
# epochs = 30
# for j in range(nets):
#     history[j] = model[j].fit(x_train,y_train, batch_size=64, epochs = epochs, 
#         validation_data = (x_val,y_val), callbacks=[annealer], verbose=0)
#     print("CNN {0}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         names[j],epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))